# Data Pre-processing

Converting raw data into ingredients fit for our strategy recipe is an annoying yet necessary work. Here's how I cleaned and processed data for this project.

Ultimately, we want these data:

1. Industry Index Price Series (Quarterly)
2. Industry Index Total Return Series (Quarterly)
3. Industry Index Earnings Series (Quarterly)

In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
from xquant.util import *

In [2]:
df_div = pd.read_csv('D:/Repositories/cicc/Industry Momentum + CAPE/data/dividends.csv', parse_dates=['announced'], dtype={'ticker':str})
df_price = pd.read_csv('D:/Repositories/cicc/Industry Momentum + CAPE/data/price.csv', index_col=['date'], parse_dates=['date'])
df_mktcap = pd.read_csv('D:/Repositories/cicc/Industry Momentum + CAPE/data/market_cap.csv', index_col=['date'], parse_dates=['date'])
df_comp = pd.read_csv('D:/Repositories/cicc/Industry Momentum + CAPE/data/WIND_index_members.csv', parse_dates=['included', 'excluded'])
df_map = pd.read_csv('D:/Repositories/cicc/Industry Momentum + CAPE/data/ticker_map.csv', index_col=['key'])
df_idx = pd.read_csv('D:/Repositories/cicc/Industry Momentum + CAPE/data/WIND_industry_index.csv', index_col=['Date'], parse_dates=['Date'])

In [3]:
# time range for back test
START = datetime(2010,1,1)
END = datetime(2020,12,31)

## Clean dividend data

In [4]:
df_div['ticker'] = df_div['ticker'].apply(add_suffix) # convert ticker symbol into standard format (e.g. 000001.SZ)

In [5]:
df_div.dropna(subset=['announced'], inplace=True)

## Clean Index Members Data

In [6]:
# map symbols to actual names of industry
df_comp['industry'] = df_comp['industry'].apply(lambda x: df_map.at[x,'value'])

In [7]:
# if stock is still a member of the index, set excluded time to a future data far away
df_comp['excluded'].fillna(pd.Timestamp('20991231'), inplace=True)

In [8]:
df_comp.dropna(subset=['included'], inplace=True)

## Calculate Financial Metrics for an Index

Very often we would need to look at certain metrics of an index, such as earnings and dividends. In a market capitalization weighted indices with $n$ members, its metric $m$ is calculated by:

$\sum^{n}_{i=1} w_{i} \cdot m_{i}$

where $w$ is the weight of member $i$ in the index (i.e. market cap of member $i$ divided by sum of market cap for all members).

### First calculate the scale (i.e. multiplier) for each industry index at a certain date

In [9]:
def get_scale(date) -> dict:
    d = dict.fromkeys(df_idx.columns,[])
    for i in df_idx.columns:
        total = 0
        f = get_index_weights(df_mktcap, df_comp.query(f"industry=='{i}'"), pd.Timestamp(date))
        for s in f.index:
            local_sum = f[s] * df_price.at[pd.Timestamp(date), s]
            if pd.notna(local_sum):
                total += local_sum
        if total != 0:            
            d[i] = df_idx.at[date,i]/total
        else:
            d[i] = np.nan
    return d

### Then calculate the quarterly weighted sum of a metric

In [12]:
def get_weighted_sum(start, end, df, sum_col):
    begin = time.time()
    days = [closest_trading_day(day, df_price.index, 'bfill') for day in quarter_generator(start,end)]
    df_sum = pd.DataFrame(index=days, columns=df_idx.columns)

    for day in df_sum.index:
        print('\r', f'Now processing: {day.date()}', end='')
        if day.quarter == 1:
            look_up = (day.year-1, 4)
        else:
            look_up = (day.year, day.quarter-1)

        scale_dict = get_scale(day)
        for industry in df_idx.columns:
            w = get_index_weights(df_mktcap, df_comp.query(f"industry=='{industry}'"), day)

            weighted = {}
            multiplier = scale_dict[industry]

            for stock in w.index:
                s = quarter_sum(ticker=stock, quarter=look_up, df=df, sum_col=sum_col, date_col='announced')
                weighted_sum = w[stock] * s
                weighted[stock] = weighted_sum

            df_sum.at[day, industry] = pd.Series(weighted, dtype=float).sum() * multiplier
    
    print(f'\ncomputation completed in {time.time()-begin} seconds.')
    return df_sum

### Quarterly Dividends

In [13]:
df_div_q = get_weighted_sum(START,END, df_div, 'div_per_share')

 Now processing: 2020-10-09
computation completed in 332.9604961872101 seconds.


In [14]:
df_div_q.head()

,Energy,Materials,Industrials,Consumer Discretionary,Consumer Staples,Health Care,Financials,Information Technology,Communication Services,Utilities,Real Estate
2010-01-04,0,0,0.0218262,0,0,0,0.222113,0.00167726,0,0,NaN
2010-04-01,0,0.053968,0.0925406,0.0043455,0.0128965,0.273544,0.0181898,0.0900128,0,0.0312638,NaN
2010-07-01,13.4659,3.63686,1.1797,0.416404,4.17079,3.38796,9.37131,1.09074,7.03341,1.35964,NaN
2010-10-08,7.01905,0.346822,0.686881,0.223824,0.814055,0.758163,2.91279,0.126117,0,2.06145,NaN
2011-01-04,0,0.0349391,0.0760834,0.00520137,0.00242668,0.168036,0,0,0,0,NaN


In [122]:
# df_div_q.to_csv('\\data\\quarterly_dividends.csv')